In [1]:
import requests
import pprint
import os

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

pp = pprint.PrettyPrinter()

In [2]:
cid = os.environ.get('SPOTIPY_CLIENT_ID')
secret = os.environ.get('SPOTIPY_CLIENT_SECRET')
username = ""
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username, scope)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

# Get favourite songs or playlist songs

In [4]:
def get_features_from_playlist(user='', playlist_id=''):
    df_result = pd.DataFrame()
    track_list = ''
    uploader_list = []
    added_ts_list = []
    artist_list = []
    title_list = []
    
    all_songs = True #As long as there is tracks not fetched from API, continue looping
    offset_index = 0
    
    if playlist_id != '' and user == '':
        print("Enter username for playlist")
        return
    
    while all_songs:
        if playlist_id == '':
            #Get songs from favourites playlist
            songs = sp.current_user_saved_tracks(limit=20, offset=offset_index)
            offset_index += 20
            if len(songs['items']) != 20:
                all_songs = False
        else:
            #Get songs from playlist of user
            lim = 20
            songs = sp.user_playlist_tracks(user, playlist_id=playlist_id, limit=lim, offset=offset_index)
            offset_index += lim
            if len(songs['items']) < lim:
                all_songs = False  
                   
        for song in songs['items']:
            #join track ids to a single string as an input parameter for audio_features function
            track_list += song['track']['id'] +','
            
            #get the time when the song was added
            added_ts_list.append(song['added_at'])
            
            #get the title of the song
            title_list.append(song['track']['name'])
            
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
            
            #get user who added song in the playlist, catering for collaboration playlists
            if playlist_id != '':
                uploader_list.append(song['added_by']['id'])
        
        #get the track features and append into a dataframe
        track_features = sp.audio_features(track_list[:-1])
        df_temp = pd.DataFrame(track_features)
        df_result = df_result.append(df_temp)
        track_list = ''
        print(df_result.shape)
    
    
    #add the timestamp added, title and artists of a song
    df_result['added_at'], df_result['song_title'], df_result['artists'] = added_ts_list, title_list, artist_list
    
    #add upload user if in a playlist
    if playlist_id != '':
        df_result['uploaded_by'] = uploader_list
                   
    return df_result

# Get audio features from playlist

In [22]:
user_playlists = sp.user_playlists(user='jkwd93')

for playlist in user_playlists['items']:
    print(playlist['id'], playlist['name'])

3wXrLUcUJTrKkViXuvDgb9 Lounge
3n2aQEZQGEnmavvfOVd3hY Chill
1KwfC9CiQVSJyMUB6qE0Xj Dance
37i9dQZF1DXdd3gw5QVjt9 Morning Acoustic
0DXoY83tBvgWkd8QH49yAI Acoustic Lounge - Chillout & Easy Listening
37i9dQZF1DWZeKCadgRdKQ Deep Focus
37i9dQZF1DX4E3UdUs7fUx Afternoon Acoustic
37i9dQZF1DWUNIrSzKgQbP Winter Acoustic


In [ ]:
df_playlist = get_features_from_playlist(user='jkwd93', playlist_id='3n2aQEZQGEnmavvfOVd3hY')

In [ ]:
df_playlist.shape

In [ ]:
df_playlist.head()

In [ ]:
df_playlist.to_hdf('chill_playlist.h5',key='df', mode='w')

# Get audio features from saved tracks

In [ ]:
df_saved_tracks = get_features_from_playlist()

In [ ]:
df_saved_tracks.shape

In [ ]:
df_saved_tracks.head()

In [ ]:
df_saved_tracks.to_hdf('saved_tracks.h5',key='df', mode='w')

# Get audio features from discover weekly

In [5]:
tastebreak = '37i9dQZF1EjaG34DgvTcv9'
discover_weekly = '37i9dQZEVXcEdB6HFaFbVC'

playlist = discover_weekly

df_playlist = get_features_from_playlist(user='jkwd93', playlist_id=playlist)

(20, 18)
(30, 18)


In [6]:
df_playlist.shape

(30, 22)

In [7]:
df_playlist.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,tempo,time_signature,track_href,type,uri,valence,added_at,song_title,artists,uploaded_by
0,0.83100,https://api.spotify.com/v1/audio-analysis/6YRL...,0.541,175053,0.213,6YRLnxlOIENOGlIFxJt000,0.000001,9,0.117,-10.291,...,60.779,4,https://api.spotify.com/v1/tracks/6YRLnxlOIENO...,audio_features,spotify:track:6YRLnxlOIENOGlIFxJt000,0.390,2019-01-13T16:00:00Z,Smile (Prod. WOOGIE),Phe Reds,
1,0.56900,https://api.spotify.com/v1/audio-analysis/0Zyt...,0.788,229530,0.269,0ZytmhLxxgXIppFb3eiPQm,0.000017,8,0.108,-11.649,...,104.875,4,https://api.spotify.com/v1/tracks/0ZytmhLxxgXI...,audio_features,spotify:track:0ZytmhLxxgXIppFb3eiPQm,0.194,2019-01-13T16:00:00Z,Gravity,Chris McClenney,
2,0.00762,https://api.spotify.com/v1/audio-analysis/05Li...,0.709,228942,0.637,05LiQlMRX2PlThoHWPjGae,0.004910,7,0.094,-8.728,...,106.988,4,https://api.spotify.com/v1/tracks/05LiQlMRX2Pl...,audio_features,spotify:track:05LiQlMRX2PlThoHWPjGae,0.377,2019-01-13T16:00:00Z,Simple,"Joe Hertz,JONES",
3,0.57100,https://api.spotify.com/v1/audio-analysis/2oUa...,0.482,311677,0.447,2oUallb0gAqnNnfGbM4qvT,0.000000,6,0.149,-7.114,...,119.790,4,https://api.spotify.com/v1/tracks/2oUallb0gAqn...,audio_features,spotify:track:2oUallb0gAqnNnfGbM4qvT,0.178,2019-01-13T16:00:00Z,X (Feat. Lee Hi),"Code Kunst,LEE HI",
4,0.04020,https://api.spotify.com/v1/audio-analysis/2R7O...,0.791,132572,0.505,2R7OVKEOGpLNVrNUPoL4jy,0.129000,2,0.293,-4.693,...,87.015,4,https://api.spotify.com/v1/tracks/2R7OVKEOGpLN...,audio_features,spotify:track:2R7OVKEOGpLNVrNUPoL4jy,0.872,2019-01-13T16:00:00Z,Wantme,Cookin Soul,


In [8]:
# df_playlist.to_hdf('discover_weekly_19012019.h5',key='df', mode='w')

In [28]:
import datetime

now = datetime.datetime.now()
ddmmyyyy = str("{:02d}".format(now.day)) + str("{:02d}".format(now.month)) + str(now.year)
savepath = 'discover_weekly_' + ddmmyyyy + '.h5'
df_playlist.to_hdf(savepath,key='df', mode='w')

'discover_weekly_19012019.h5'